## Необходимые библиотеки

In [ ]:
import numpy as np
import seaborn as sns #надо установить pip3 install seaborn
import pyarrow.parquet as pq
from sklearn.preprocessing import LabelEncoder

import os
import boto3

: 

## Загрузка данных

In [ ]:
bucket_name = 'mldata'

# адрес сервера, логин и пароль беруться из пременных окружения 
# переменные окружения уже установлены AWS_ENDPOINT_URL, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY
s3_target = boto3.resource('s3', 
    endpoint_url=os.environ["AWS_ENDPOINT_URL"],
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)

# создание корзины
s3_target.create_bucket(Bucket=bucket_name)

In [ ]:
train_file = 'input_train.parquet'
# чтение файла с s3 файла train_file в локальный файл /tmp/data.parquet
s3_target.Bucket(bucket_name).download_file(train_file, '/tmp/data.parquet')


In [ ]:
train_file = '/tmp/data.parquet'
df=pq.read_table(train_file)
df=df.to_pandas()

## Типы данных
Есть столбец с типом datetime, object и с численными значениями

In [ ]:
df.dtypes

Посмотрим на данные в наборе данных:

In [ ]:
df.head()

Определим размер набора данных

In [ ]:
df.shape

## Кодировка данных
Закодриуем src_ip

In [ ]:
labelencoder = LabelEncoder()
df['src_ip_id'] = labelencoder.fit_transform(df['src_ip'])

## Основные статистические характеристики набора данных:

Для категориальных признаков этот метод показывает: 

    - unique количество уникальных значений;
    - top значения;
    - freq - частота появления значений.

Для численных значений этот метод показывает:

    - count количество строк в столбце;
    - mean среднее значение столбца;
    - std стандартное отклонение;
    - min/max минимальные и максимальные значения;
    - 25%, 50% и 75% граница каждого квартиля. 
    

In [ ]:
df.describe(include = "all")

Для дискретных значений как день недели, день месяца, час, такой показатель как среднее значение мало информативен. Для этого вычислим такие показатели как медиана (серединное значение) и мода (часто встречающеся значение)

Медиана

Для daymonth, dayweek, hour и src_ip_id медиана вышла приблизительно в середине допустимых значений. Тогда как для dst_cnt половина значений в датасете приходятся на количество соединений от 1 до 2


In [ ]:
df[['daymonth','dayweek','hour','dst_cnt','src_ip_id']].median()

Мода

И здесь также видно, что чаще всего встречается значение для dst_cnt=1

In [ ]:
df[['daymonth','dayweek','hour','dst_cnt','src_ip_id']].mode()

Проверим уникальность данных

In [ ]:
key = df.columns

df_dedupped2 = df.drop_duplicates(subset=key)

print('Кол-во значений ',df.shape[0])
print('Кол-во уникальных значений ',df_dedupped2.shape[0])

Проверим пустые значения

In [ ]:
# определяем цвета 
# желтый - пропущенные данные, синий - не пропущенные
df.isnull().sum()
colours = ['#000099', '#ffff00'] 
sns.heatmap(df.isnull(), cmap=sns.color_palette(colours))
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

## Визуальное исследование датасета

Оценим распределение признаков

Интерпретация результатов:
1) dst_cnt - как и показали ранее значения медианы и моды, большой перевес в значениях 1 и 2.
2) daymonth - ожидаем значения от 1 до 31, виден пропуск данных с 15 по 20.
3) dayweek - ожидаем значения от 1 до 7, все значения присутствуют
4) hour - ожидаем значения от 0 до 23, все значения присутствуют
5) src_ip_id - по некоторым ip адресам мало значений

In [ ]:
sns.distplot(df["dst_cnt"])

In [ ]:
sns.distplot(df["daymonth"])

In [ ]:
sns.distplot(df["dayweek"])

In [ ]:
sns.distplot(df["hour"])

In [ ]:
sns.distplot(df["src_ip_id"])

Посмотрим на количество подключений в течение месяца

In [ ]:
sns.jointplot(x="src_ip_id", y="dst_cnt", data=df);

In [ ]:
sns.jointplot(x="daymonth", y="dst_cnt", data=df);

In [ ]:
sns.jointplot(x="dayweek", y="dst_cnt", data=df);

In [ ]:
sns.jointplot(x="hour", y="dst_cnt", data=df);

Построим парные диаграммы по&nbsp;всем показателям по&nbsp;исходному набору данных:

Интерпретация
1) также видны пропуски данных
2) видны выбросы в количестве подключений по дням месяца, недели и часам, по src_ip

In [ ]:
sns.pairplot(df, plot_kws=dict(linewidth=0));

## Информация о корреляции признаков

Построим корреляционную матрицу по всему набору данных:

In [ ]:
df.corr()

Визуализируем корреляционную матрицу с&nbsp;помощью тепловой карты:

Наибольшая зависимость (хотя и очень маленькая) между src_ip_id и dst_cnt

In [ ]:
sns.heatmap(df.corr(), annot=True, fmt=".2f");